In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
hrz_loc = pd.read_csv('./data/horizons/florian/Top_Tuxen_Valdemar_TWT',names = ['cdp_x', 'cdp_y','twt'], sep='\s+')

In [ ]:
hrz = pd.read_csv('./data/horizons/florian/LOCRETA surfaces meters/TopTuxen_Florian (Depth 1)',names = ['cdp_x', 'cdp_y','twt','IL','XL'], sep='\s+', skiprows=20)

In [ ]:
hrz

In [ ]:
hrz = hrz.loc[(hrz['cdp_x'] < hrz_loc['cdp_x'].max() ) & (hrz['cdp_y'] < hrz_loc['cdp_y'].max()) ]

In [ ]:
hrz

In [ ]:
wells = pd.read_excel('./data/DK_wells_locations.xlsx', names = ['Well', 'Seq.No', 'Field', 'Structure', 'Sector', 'Platform', 'UTM x', 'UTM y', 'UTM Zone', 'Datum'])

In [ ]:
wells.drop(index = wells.loc[wells['Sector'] == 'GREENLAND'].index, inplace=True)
wells.drop(columns = 'Datum', inplace=True)

In [ ]:
wells.dropna(axis=0, inplace=True)

In [ ]:
wells

In [ ]:
from scipy.interpolate import griddata
xi = np.linspace(hrz.cdp_x.min(), hrz.cdp_x.max(), 2500)
yi = np.linspace(hrz.cdp_y.min(), hrz.cdp_y.max(), 2500)
X, Y = np.meshgrid(xi, yi)
Z = griddata((hrz.cdp_x, hrz.cdp_y), hrz.twt, (X, Y))


In [ ]:
wells.reset_index(drop=True, inplace = True)

In [ ]:
import matplotlib
plt.style.use('default')
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['font.size'] = 16
matplotlib.rcParams['legend.fontsize'] = 'small'
matplotlib.rcParams['axes.labelsize'] = 'medium'
plt.rc('xtick',labelsize=12)
plt.rc('ytick',labelsize=12)

In [ ]:
import shapefile

In [ ]:
#UTM 32 N (ED 50)
fields_poly = shapefile.Reader('./data/Fields_ED50_UTM31n_SL.shp')
geomet = fields_poly.shapeRecords()

In [ ]:
plt.figure()
for p in range(len(geomet)):
        plt.plot(np.array(geomet[p].shape.points)[:,0], np.array(geomet[p].shape.points)[:,1], lw = 1 ,color='k')

In [ ]:
plt.imshow(Z, aspect='equal', extent = [hrz.cdp_x.min(), hrz.cdp_x.max() , hrz.cdp_y.min(), hrz.cdp_y.max()], cmap='Spectral', interpolation='spline36', alpha=.5, origin ='lower')


In [ ]:
from matplotlib_scalebar.scalebar import ScaleBar
#plt.figure()
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(Z, aspect='equal', extent = [hrz.cdp_x.min(), hrz.cdp_x.max() , hrz.cdp_y.min(), hrz.cdp_y.max()], cmap='Spectral', interpolation='spline36', alpha=.5, origin ='lower')

plt.xlim(hrz_loc.cdp_x.min(), hrz_loc.cdp_x.max())
plt.ylim(hrz_loc.cdp_y.min(), hrz_loc.cdp_y.max()-100)

cbar = plt.colorbar()
cbar.set_label(label='Depth (m)', rotation = -90, labelpad = 18)
plt.clim(2300, 3200) 

cs = plt.contour(Z,levels=10 , extent=[hrz.cdp_x.min(), hrz.cdp_x.max() , hrz.cdp_y.min(), hrz.cdp_y.max()], colors='black', linewidths=2, alpha=0.7)
plt.clabel(cs, inline=True, fmt='%1.0f')




plt.scatter(wells['UTM x'] , wells['UTM y'], marker='^',edgecolor='k', s=200, color='limegreen', label='Well')


props = dict(boxstyle='square', facecolor='white', alpha=1, )
#for i, txt in enumerate(wells['Well']):
#    plt.annotate(txt, (wells['UTM x'][i]-1500, wells['UTM y'][i]+530), bbox=props,fontsize =8)

plt.annotate('Valdemar', (595000, 6193250), bbox = props, fontsize = 24)
plt.annotate('Boje', (604000, 6193200), bbox = props, fontsize = 24)
plt.annotate('Roar', (604000, 6185100), bbox = props, fontsize = 24)


plt.annotate('North Jens', (596500, 6189600), bbox = dict(boxstyle='square', facecolor='white', alpha=.75, ), fontsize = 14)

plt.annotate('Bo', (596000, 6185600), bbox = dict(boxstyle='square', facecolor='white', alpha=.75, ), fontsize = 14)

plt.annotate('Bo South', (597700, 6180000), bbox = dict(boxstyle='square', facecolor='white', alpha=.75, ), fontsize = 14)


scalebar = ScaleBar(1, "m", length_fraction=0.25, box_alpha=0.8,location=3)
ax.add_artist(scalebar)
    
plt.xlabel('UTM Easting (m)')
plt.ylabel('UTM Northing (m)')
#plt.legend(loc=4, fontsize=16)
plt.title('Top Tuxen Depth Map', fontsize=28);


for p in range(len(geomet)):
    if p == 36:
        plt.plot(np.array(geomet[p].shape.points)[:,0], np.array(geomet[p].shape.points)[:,1], lw = 2 ,color='k',label = 'Field delineations')
    else:
        plt.plot(np.array(geomet[p].shape.points)[:,0], np.array(geomet[p].shape.points)[:,1], lw = 2 ,color='k')
plt.legend(loc=4, fontsize=12)        
        
#fig_path = 'M:/LOCRETA_WORKPACKAGE_D/Valdemar_sprint_well_data_Mads_L/MLO documents/Figures/'
#plt.savefig(fig_path+'Tuxen_surveymap.png',format='png', bbox_inches='tight', dpi=150)#